In [3]:
from pyomo.core import *
from pyomo.opt import SolverFactory, SolverManagerFactory
import pyomo.environ
import pandas as pd
import numpy as np
from itertools import compress

In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:80% !important; }</style>")

In [4]:
class InputData:
    
    #----------------------------------------------------------------------
    
    def __init__(self, ExcelPath, demands='Demand data',solar='Solar data', tech='Technology', gen='General'):
        self.path = ExcelPath
        self.DemandSheet = demands
        self.SolarSheet = solar
        self.TechSheet = tech
        self.GeneralSheet = gen
        
        self.TechOutputs = None
        self.Technologies = None
        self.DemandData = None
        self.StorageData = None
        
        self.Initialize()
        
    def Initialize(self):
        self.TechParameters()
        self.TechOutput()
        self.Demanddata()
        self.Storage()
        
    def TechParameters(self):
        Technologies=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=1, index_col=0, skip_footer=38) #technology characteristics
        Technologies=Technologies.dropna(axis=1, how='all') #technology characteristics
        Technologies=Technologies.fillna(0) #technology characteristics
        self.Technologies=Technologies
    
    def TechOutput(self):
        TechOutputs=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=15, index_col=0, skip_footer=26) #Output matrix
        TechOutputs=TechOutputs.dropna(axis=0,how='all')  #Output matrix
        TechOutputs=TechOutputs.dropna(axis=1,how='all')  #Output matrix
        self.TechOutputs=TechOutputs
        
    def Demanddata(self):
        DemandDatas=pd.read_excel(self.path,sheetname=self.DemandSheet)
        self.DemandData=DemandDatas
        
    def Storage(self):
        Storage=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=40, index_col=0, skip_footer=0) #
        Storage=Storage.dropna(axis=1, how='all')
        Storage=Storage.fillna(0)
        self.StorageData=Storage
        
    
    
    #----------------------------------------------------------------------

    def Dict1D(self,dictVar,dataframe):
        for i,vali in enumerate(dataframe.index):
            dictVar[i+1]=round(dataframe.iloc[i][1],4)
        return dictVar

    def Dict1D_val_index(self,dictVar,dataframe):
        for i,vali in enumerate(dataframe.index):
            dictVar[vali]=round(dataframe.iloc[i][1],4)
        return dictVar

    def DictND(self,dictVar,dataframe):
        for i,vali in enumerate(dataframe.index):
            for j,valj in enumerate(dataframe.columns):
                dictVar[vali,valj]=round(dataframe.iloc[i][j+1],4)
        return dictVar
    
    
    
    
    #----------------------------------------------------------------------
    
    def Demands(self):
        Loads_init=self.DemandData
        Loads_init.index=list(range(1,len(Loads_init.index)+1))
        Loads_init.columns=list(range(1,len(Loads_init.columns)+1))
        loads_init={}
        loads_init=self.DictND(loads_init,Loads_init)
        return loads_init
    
    
    
    
    #----------------------------------------------------------------------
    
    def SolarData(self):
        SolarData=pd.read_excel(self.path,sheetname=self.SolarSheet)
        SolarData.columns=[1]
        solar_init={}
        solar_init=self.Dict1D(solar_init,SolarData)
        return solar_init
    
    
    
    
    #----------------------------------------------------------------------
    
    def CHP_list(self):
        Dispatch_tech=pd.DataFrame(self.TechOutputs.sum(0)) #find dispatch tech (CHP)
        CHP_setlist=[]
        for n,val in enumerate(Dispatch_tech[0]):
            if val>1:
                CHP_setlist.append(n+2) #first is electricity +1 since it starts at 0
        return CHP_setlist
    
    
    
    
    #----------------------------------------------------------------------
    
    def Roof_tech(self):
        Roof_techset=[]
        for n,val in enumerate(self.Technologies.loc["Area (m2)"]):
            if val>0:
                Roof_techset.append(n+2) #first is electricity +1 since it starts at 0
                
        return Roof_techset
    
    
    
    
    #----------------------------------------------------------------------
        
    def cMatrix(self):
        
        #Based on the + - values, prepare data for generating coupling matrix        
        TechOutputs2=self.TechOutputs.multiply(np.array(self.Technologies.loc['Efficiency (%)']))
        TechOutputs2.loc[TechOutputs2.index!='Electricity']=TechOutputs2.loc[TechOutputs2.index!='Electricity'].multiply(np.array(self.Technologies.loc['Output ratio'].fillna(value=1).replace(0,1)))
        TechOutputs2[TechOutputs2<0]=-1

        addGrid=np.zeros(len(self.DemandData.columns),)
        addGrid[0]=1 #add electricity to coupling matrix
        Grid=pd.DataFrame(addGrid,columns=["Grid"],index=self.DemandData.columns).transpose()
        
        Cmatrix=TechOutputs2.transpose()
        Cmatrix=pd.concat([Grid,Cmatrix])
        Cmatrix.index=list(range(1,len(TechOutputs2.columns)+2))
        Cmatrix.columns=list(range(1,len(TechOutputs2.index)+1))
        cMatrixDict={}
        cMatrixDict=self.DictND(cMatrixDict,Cmatrix)
        
        return cMatrixDict
    
    
    
    
    #----------------------------------------------------------------------
    
    def PartLoad(self):
        PartLoad=self.Technologies.loc["MinLoad (%)",]/100

        partload=self.TechOutputs.iloc[0:1].mul(list(PartLoad),axis=1)
        partload=pd.concat([partload,self.TechOutputs.iloc[1:].mul(list(PartLoad),axis=1)], axis=0)
        partload=partload.abs()
        partload=partload.transpose()
        partload.index=list(range(1+1,len(self.TechOutputs.columns)+2))
        partload.columns=list(range(1,len(self.TechOutputs.index)+1))
        SolartechsSets=list(compress(list(range(1+1,len(self.Technologies.columns)+2)), list(self.Technologies.loc["Area (m2)"]>0)))

        for i in SolartechsSets:
            partload.drop(i, inplace=True)

        PartloadInput={}
        PartloadInput=self.DictND(PartloadInput,partload)
        
        return PartloadInput
    
    
    
    
    #----------------------------------------------------------------------
    
    def MaxCapacity(self):
        MaxCap=self.Technologies.loc["Maximum Capacity",]
        MaxCap.index=list(range(2,len(self.Technologies.loc["MinLoad (%)",].index)+2))
        MaxCap.round(decimals=3)
        maxCap= MaxCap.to_dict()
        
        SolartechsSets=list(compress(list(range(1+1,len(self.Technologies.columns)+2)), list(self.Technologies.loc["Area (m2)"]>0)))
        for i in SolartechsSets:
            maxCap.pop(i, None)
            
        return maxCap
     
    
    
    
    #----------------------------------------------------------------------
    
    def SolarSet(self):
        return list(compress(list(range(1+1,len(self.Technologies.columns)+2)), list(self.Technologies.loc["Area (m2)"]>0)))
    
    
    def DispTechsSet(self):
        return list(compress(list(range(1+1,len(self.Technologies.columns)+2)), list(self.Technologies.loc["Area (m2)"]==0)))

    
    def partloadtechs(self):
        PartLoad=self.Technologies.loc["MinLoad (%)",]/100

        partload=self.TechOutputs.iloc[0:1].mul(list(PartLoad),axis=1)
        partload=pd.concat([partload,self.TechOutputs.iloc[1:].mul(list(PartLoad),axis=1)], axis=0)
        partload=partload.abs()
        partload=partload.transpose()
        partload.index=list(range(1+1,len(self.TechOutputs.columns)+2))
        partload.columns=list(range(1,len(self.TechOutputs.index)+1))
        SolartechsSets=list(compress(list(range(1+1,len(self.Technologies.columns)+2)), list(self.Technologies.loc["Area (m2)"]>0)))

        for i in SolartechsSets:
            partload.drop(i, inplace=True)
            
        return list(partload.loc[partload.sum(axis=1)>0].index)
    
    
    #----------------------------------------------------------------------
    
    def LinearCost(self):
        LinearCost=self.Technologies.loc["CapCost (chf/kW)",]

        linCost=self.TechOutputs.iloc[0:1].mul(list(LinearCost),axis=1)
        linCost=pd.concat([linCost,self.TechOutputs.iloc[1:].mul(list(LinearCost),axis=1)], axis=0)

        linCost=linCost.transpose()
        for name in linCost.columns[1:]:
            linCost.loc[linCost["Electricity"] >1, name] = 0

        linCost.loc[linCost["Electricity"] <0, "Electricity"]=0
        linCost=linCost.abs()
        
        addGrid=np.zeros(len(self.DemandData.columns),)
        addGrid[0]=1 #add electricity to coupling matrix
        Grid=pd.DataFrame(addGrid,columns=["Grid"],index=self.DemandData.columns).transpose()
        
        linCost=pd.concat([Grid,linCost])

        linCost.index=list(range(1,len(self.TechOutputs.columns)+2))
        linCost.columns=list(range(1,len(self.TechOutputs.index)+1))
        linCost.loc[1]=0

        linCapCosts={}
        linCapCosts=self.DictND(linCapCosts,linCost)
        
        return linCapCosts
     
    
    
    
    #----------------------------------------------------------------------
    ### Find which is the primary input for capacity #####
    def DisDemands(self):
        
        CHPlist=self.CHP_list()
        dispatch_demands=np.zeros((len(CHPlist), 2), dtype=int)

        for n,val in enumerate(CHPlist):
            counter=0
            for i, value in enumerate(np.array(self.TechOutputs[[val-2]],dtype=int)):
                if value[0]>0 and counter==0:
                    dispatch_demands[n,0]=i+1
                    counter=1
                if value[0]>0 and counter==1:
                    dispatch_demands[n,1]=i+1

        return dispatch_demands    
    
    
    
    #----------------------------------------------------------------------
    
    def InterestRate(self):
        Interest_rate=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=8, index_col=0, skip_footer=7) #
        Interest_rate=Interest_rate.dropna(axis=1,how='all')
        Interest_rate_R=Interest_rate.loc["Interest Rate r"][0]
        return Interest_rate_R
    
    
    
    
    #----------------------------------------------------------------------
    
    def LifeTime(self):
        Life=pd.DataFrame(list(self.Technologies.loc["Lifetime (yr)"]))
        Life.columns=[1]
        Life.index=list(range(2,len(self.TechOutputs.columns)+2))
        lifeTimeTechs={}
        lifeTimeTechs=self.Dict1D_val_index(lifeTimeTechs, Life)
        return lifeTimeTechs
        
    
    
    
    #----------------------------------------------------------------------
        
    def NPV(self):
        Interest_rate_R=self.InterestRate()
        Life=pd.DataFrame(list(self.Technologies.loc["Lifetime (yr)"]))
        Life.columns=[1]
        Life.index=list(range(2,len(self.TechOutputs.columns)+2))
        
        NetPresentValue=1 / (((1 + Interest_rate_R) ** Life - 1) / (Interest_rate_R * ((1 + Interest_rate_R) ** Life)))
        NetPresentValueTech={}
        NetPresentValueTech=self.Dict1D_val_index(NetPresentValueTech,NetPresentValue)
        return NetPresentValueTech
        
    
    
    
    #----------------------------------------------------------------------
    
    def VarMaintCost(self):
        VarOMF=pd.DataFrame(list(self.Technologies.loc["OMVCost (chf/kWh)"]))
        VarOMF.columns=[1]
        VarOMF.index=list(range(1+1,len(self.TechOutputs.columns)+2))
        VarOMF.loc[1]=0
        omvCosts={}
        omvCosts=self.Dict1D_val_index(omvCosts, VarOMF)
        return omvCosts
        
    
    
    
    #----------------------------------------------------------------------
    
    def CarbFactors(self):
        Carbon=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=24, index_col=0, skip_footer=16) #
        Carbon=Carbon.dropna(axis=0,how='all')
        Carbon=Carbon.dropna(axis=1,how='all')
        Carbon.index=[1]

        ElectricityCF=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=1, index_col=0, skip_footer=14) #
        ElectricityCF=ElectricityCF.dropna(axis=0,how='all')
        ElectricityCF=ElectricityCF.dropna(axis=1,how='all')
        del ElectricityCF["Price (chf/kWh)"]
        ElectricityCF.index=[1]

        CarbonFactors= pd.concat([ElectricityCF, Carbon], axis=1)
        CarbonFactors.columns=list(range(1,len(CarbonFactors.columns)+1))
        CarbonFactors=CarbonFactors.transpose()

        carbonFactors={}
        carbonFactors=self.Dict1D(carbonFactors,CarbonFactors)
        
        return carbonFactors
       
    
    
    
    #----------------------------------------------------------------------
    
    def FuelPrice(self):
        Fuel=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=1, index_col=0, skip_footer=10) #
        Fuel=Fuel.dropna(axis=0,how='all')

        Carbon=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=24, index_col=0, skip_footer=16) #
        Carbon=Carbon.dropna(axis=0,how='all')
        Carbon=Carbon.dropna(axis=1,how='all')
        Carbon.index=[1]

        ElectricityCF=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=1, index_col=0, skip_footer=14) #
        ElectricityCF=ElectricityCF.dropna(axis=0,how='all')
        ElectricityCF=ElectricityCF.dropna(axis=1,how='all')
        del ElectricityCF["Price (chf/kWh)"]
        ElectricityCF.index=[1]

        CarbonFactors= pd.concat([ElectricityCF, Carbon], axis=1)
        CarbonFactors.columns=list(range(1,len(CarbonFactors.columns)+1))
        CarbonFactors=CarbonFactors.transpose()

    
        for n,val in enumerate(Fuel["CO2 (kg/kWh)"]):
            for index,value in CarbonFactors.iterrows():
                if float(val)==float(value):
                    CarbonFactors.loc[index]=float(Fuel["Price (chf/kWh)"][n])

        opPrices={}
        opPrices=self.Dict1D(opPrices,CarbonFactors)
        
        return opPrices
     
    
    
    
    #----------------------------------------------------------------------
    
    def FeedIn(self):
        Tariff=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=11, index_col=0, skip_footer=0) #
        Tariff=Tariff.dropna(axis=0,how='all')
        Tariff=Tariff.dropna(axis=1,how='all')
        Tariff.columns=[1]
        Tariff.index=list(range(1,len(self.DemandData.columns)+1))

        feedInTariffs={}
        feedInTariffs=self.Dict1D_val_index(feedInTariffs,Tariff)
        return feedInTariffs
    
    
    
    #### Storage ####
    #----------------------------------------------------------------------
    
    def StorageCh(self):
        maxStorCh={}
        MaxCharge=pd.DataFrame(list(self.StorageData.loc["max_charge"]))
        MaxCharge.columns=[1]
        maxStorCh=self.Dict1D(maxStorCh, MaxCharge)
        return maxStorCh
    
    
    def StorageDisch(self):
        maxStorDisch={}
        MaxDischarge=pd.DataFrame(list(self.StorageData.loc["max_discharge"]))
        MaxDischarge.columns=[1]
        maxStorDisch=self.Dict1D(maxStorDisch, MaxDischarge)
        return maxStorDisch
    
    
    def StorageLoss(self):
        lossesStorStanding={}
        losses=pd.DataFrame(list(self.StorageData.loc["decay"]))
        losses.columns=[1]
        lossesStorStanding=self.Dict1D(lossesStorStanding, losses)
        return lossesStorStanding
    
    
    def StorageEfCh(self):
        chargingEff={}
        Ch_eff=pd.DataFrame(list(self.StorageData.loc["ch_eff"]))
        Ch_eff.columns=[1]
        chargingEff=self.Dict1D(chargingEff, Ch_eff)
        return chargingEff
    
    def StorageEfDisch(self):
        dischLosses={}
        Disch_eff=pd.DataFrame(list(self.StorageData.loc["disch_eff"]))
        Disch_eff.columns=[1]
        dischLosses=self.Dict1D(dischLosses, Disch_eff)
        return dischLosses
    
    
    def StorageMinSoC(self):
        minSoC={}
        min_state=pd.DataFrame(list(self.StorageData.loc["min_state"]))
        min_state.columns=[1]
        minSoC=self.Dict1D(minSoC, min_state)
        return minSoC
    
    
    def StorageLife(self):
        lifeTimeStorages={}
        LifeBattery=pd.DataFrame(list(self.StorageData.loc["LifeBat (year)"]))
        LifeBattery.columns=[1]
        lifeTimeStorages=self.Dict1D(lifeTimeStorages, LifeBattery)
        return lifeTimeStorages
    
    
    def StorageLinCost(self):
        linStorageCosts={}
        LinearCostStorage=pd.DataFrame(list(self.StorageData.loc["CostBat (chf/kWh)"]))
        LinearCostStorage.columns=[1]
        linStorageCosts=self.Dict1D(linStorageCosts, LinearCostStorage)
        return linStorageCosts
    
    
    def StorageNPV(self):
        Interest_rate_R=self.InterestRate()
        LifeBattery=pd.DataFrame(list(self.StorageData.loc["LifeBat (year)"]))
        LifeBattery.columns=[1]
        NetPresentValueStor={}
        NetPresentValueStorage=1 / (((1 + Interest_rate_R) ** LifeBattery - 1) / (Interest_rate_R * ((1 + Interest_rate_R) ** LifeBattery)))
        NetPresentValueStor=self.Dict1D(NetPresentValueStor,NetPresentValueStorage)
        return NetPresentValueStor

     

In [5]:
excel_path=r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\General_input.xlsx'
data=InputData(excel_path)


In [6]:
data.MaxCapacity().keys()

dict_keys([2, 3, 4, 7, 8, 9])

In [167]:
#-----------------------------------------------------------------------------#
## Creating a model ##
#-----------------------------------------------------------------------------#

model = ConcreteModel()

# sets definition
model.Time = RangeSet(1, data.DemandData.shape[0])
model.SubTime = RangeSet(2, data.DemandData.shape[0], within=model.Time)
model.In = RangeSet(1, data.Technologies.shape[1]+1)
model.W_Ogrid = Set(initialize=list(range(1+1,len(data.Technologies.columns)+2)), within=model.In) #techs w/o grid


model.Out = RangeSet(1, data.DemandData.shape[1])
number_of_demands= list(range(1, data.DemandData.shape[1]+1))

model.SolarTechs = Set(initialize=data.SolarSet(), within=model.In)

model.DispTechs = Set(initialize=data.DispTechsSet(), within=model.In)
model.Techs = Set(initialize=list(data.MaxCapacity().keys()), within=model.In)
model.PartLoad=Set(initialize=data.partloadtechs(), within=model.In)

model.CHP = Set(initialize=data.CHP_list(), within=model.In) # set dispatch tech set
model.roof_tech = Set(initialize=data.Roof_tech(), within=model.In) # set tech with roof area set
model.gas_tech = Set(initialize=[3,4,7,8], within=model.In) #




# coupling matrix & Technical parameters
model.cMatrix = Param(model.In, model.Out, initialize=data.cMatrix())                                      # coupling matrix technology efficiencies 
model.maxCapTechs = Param(model.DispTechs, initialize=data.MaxCapacity())
model.maxCapTechsAll = Param(model.Techs, initialize=data.MaxCapacity())
model.maxStorCh = Param(model.Out, initialize=data.StorageCh())
model.maxStorDisch = Param(model.Out, initialize= data.StorageDisch())

model.lossesStorStanding = Param(model.Out, initialize = data.StorageLoss())
model.chargingEff = Param(model.Out, initialize = data.StorageEfCh())
model.dischLosses = Param(model.Out, initialize = data.StorageEfDisch())
model.minSoC = Param(model.Out, initialize = data.StorageMinSoC())
model.partLoad = Param(model.In, model.Out, initialize=data.PartLoad()) #PartloadInput
model.maxSolarArea = Param(initialize=500)

# carbon factors
model.carbonFactors = Param(model.In, initialize=data.CarbFactors())
model.maxCarbon = Param(initialize=650000)

# Cost parameters
model.linCapCosts = Param(model.In, model.Out, initialize= data.LinearCost())         # Technologies capital costs
model.linStorageCosts = Param(model.Out, initialize = data.StorageLinCost())
model.opPrices = Param(model.In, initialize=data.FuelPrice())                       # Operating prices technologies
model.feedInTariffs = Param(model.Out, initialize=data.FeedIn())               # feed-in tariffs
model.omvCosts = Param(model.In, initialize=data.VarMaintCost())                            # Maintenance costs
model.interestRate = Param(within=NonNegativeReals, initialize=data.InterestRate())
# lifetime
model.lifeTimeTechs = Param(model.In, initialize = data.LifeTime())
model.lifeTimeStorages = Param(model.Out, initialize = data.StorageLife())


## Declaring Global Parameters ##
model.timeHorizon = Param(within=NonNegativeReals, initialize=20)
model.bigM = Param(within=NonNegativeReals, initialize=5000)

#loads

model.loads = Param(model.Time, model.Out, initialize=data.Demands())
model.solarEm = Param(model.Time, initialize=data.SolarData())


## Global variables
model.P = Var(model.Time, model.In, domain=NonNegativeReals)
model.Pexport = Var(model.Time, model.Out, domain=NonNegativeReals)
model.Capacities = Var(model.In, model.Out, domain=NonNegativeReals)
model.Ytechnologies = Var(model.In, model.Out, domain=Binary)
model.Yon = Var(model.Time, model.In, domain=Binary)
model.TotalCost = Var(domain=Reals)
model.OpCost = Var(domain=NonNegativeReals)
model.MaintCost = Var(domain=NonNegativeReals)
model.IncomeExp = Var(domain=NonNegativeReals)
model.InvCost = Var(domain=NonNegativeReals)
model.TotalCarbon = Var(domain=Reals)
model.TotalCarbon2 = Var(domain=Reals)
model.NetPresentValueTech = Param(model.In, domain=NonNegativeReals, initialize=data.NPV())
model.NetPresentValueStor = Param(model.Out, domain=NonNegativeReals, initialize=data.StorageNPV())

                      
#Storage variables
model.Qin = Var(model.Time, model.Out, domain=NonNegativeReals)
model.Qout = Var(model.Time, model.Out, domain=NonNegativeReals)
model.E = Var(model.Time, model.Out, domain=NonNegativeReals)
model.StorageCap = Var(model.Out, domain=NonNegativeReals)
model.Ystorage = Var(model.Out, domain=Binary)
#model.maxStorageCap = Param(model.Out, initialize= maxStorageCap)

In [168]:
#-----------------------------------------------------------------------------#
## GLobal constraints
#-----------------------------------------------------------------------------#
def loadsBalance_rule(model, t, out):
    return (model.loads[t,out] + model.Pexport[t,out] <= (model.Qout[ t,out] - model.Qin[t,out] + 
                                                        sum(model.P[t,inp]*model.cMatrix[inp,out] for inp in model.In)))
model.loadsBalance = Constraint(model.Time, model.Out, rule=loadsBalance_rule)

def capacityConst_rule(model, t, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, inp]  * model.cMatrix[inp,out]<= model.Capacities[inp,out])
model.capacityConst = Constraint(model.Time, model.In, model.Out, rule=capacityConst_rule)

def maxCapacity_rule2(model, tech, out):
    return (model.Capacities[tech, out] <= model.maxCapTechs[tech])
model.maxCapacity2 = Constraint(model.DispTechs, model.Out, rule=maxCapacity_rule2)



def capacity_rule(model, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(model.Capacities[inp,out] == 0)
    else:
        return(Constraint.Skip)
model.capacity_feasibility = Constraint(model.In, model.Out, rule=capacity_rule)


#def maxCapacityBIS_rule(model):
#    return (model.Capacities[2, 2] == 10)
#model.maxCapacityBIS = Constraint(rule=maxCapacityBIS_rule)

'''
def carbonConst_rule(model):
    return (model.TotalCarbon <= model.maxCarbon)
model.carbonConst = Constraint(rule=carbonConst_rule)
'''

'\ndef carbonConst_rule(model):\n    return (model.TotalCarbon <= model.maxCarbon)\nmodel.carbonConst = Constraint(rule=carbonConst_rule)\n'

In [169]:
#-----------------------------------------------------------------------------#
## Specific constraints 
#-----------------------------------------------------------------------------#

def partLoadL_rule(model, t, disp, out):  
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.partLoad[disp, out] * model.Capacities[disp, out] <= 
                                                                    (model.P[disp, out] * model.cMatrix[disp,out] 
                                                                    + model.bigM * (1 - model.Yon[t, disp])))
model.partLoadL = Constraint(model.Time, model.PartLoad, model.Out, rule=partLoadL_rule)

def partLoadU_rule(model, t, disp, out):    
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, disp] * model.cMatrix[disp, out] <= model.bigM * model.Yon[t, disp])
model.partLoadU = Constraint(model.Time, model.PartLoad, model.Out, rule=partLoadU_rule)



def solarInput_rule(model, t, sol, out):
    if model.cMatrix[sol, out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, sol] == model.solarEm[t] * model.Capacities[sol, out])
model.solarInput = Constraint(model.Time, model.SolarTechs, model.Out, rule=solarInput_rule) 


def roofArea_rule(model,roof, demand):
    return (sum(model.Capacities[roof,d] for d in number_of_demands) <= model.maxSolarArea) #DemandData.shape[1]
model.roofArea = Constraint(model.roof_tech,model.Out,rule=roofArea_rule) #model.roof_tech

def fixCostConst_rule(model, inp, out):
    return (model.Capacities[inp,out] <= model.bigM * model.Ytechnologies[inp,out])
model.fixCostConst = Constraint(model.In, model.Out, rule=fixCostConst_rule)

dispatch_demands=data.DisDemands()
CHP_list=data.CHP_list()
model.con = ConstraintList()
for x in range(1, len(dispatch_demands)+1):
    model.con.add(model.Capacities[CHP_list[x-1],dispatch_demands[x-1,1]] == model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,1]] / model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,0]] * model.Capacities[CHP_list[x-1],dispatch_demands[x-1,0]])
    model.con.add(model.Ytechnologies[CHP_list[x-1],dispatch_demands[x-1,0]]==model.Ytechnologies[CHP_list[x-1],dispatch_demands[x-1,1]])
    model.con.add(model.Capacities[CHP_list[x-1],dispatch_demands[x-1,0]] <= model.maxCapTechs[CHP_list[x-1]] * model.Ytechnologies[CHP_list[x-1],dispatch_demands[x-1,0]])

#model.con.construct()

In [170]:
#-----------------------------------------------------------------------------#
## Storage constraints
#-----------------------------------------------------------------------------#

def storageBalance_rule(model, t, out):
    return (model.E[t, out] == (model.lossesStorStanding[out] * model.E[(t-1), out] 
                                + model.chargingEff[out] * model.Qin[t, out] 
                                - (1/model.dischLosses[out]) * model.Qout[t, out]))
model.storageBalance = Constraint(model.SubTime, model.Out, rule=storageBalance_rule)
'''
model.StorCon = ConstraintList()
for x in range(1, DemandData.shape[1]+1):
    
    #model.StorCon.add(model.E[1, x] == model.StorageCap[x] * model.minSoC[x])
    model.StorCon.add(model.E[1, x] == model.E[8760, x])
    #model.StorCon.add(model.Qout[1, x] == 0)
'''    

'''
def storageInitBattery_rule(model):
    return (model.E[1, 1] == model.StorageCap[1] * model.minSoC[1])
model.storageInitBattery = Constraint(rule=storageInitBattery_rule)

def storageInitThermal1_rule(model):
    return (model.E[1, 2] == model.E[8760, 2])
model.storageInitThermal1 = Constraint(rule=storageInitThermal1_rule)

def storageInitThermal2_rule(model):
    return (model.Qout[1, 2] == 0)
model.storageInitThermal2 = Constraint(rule=storageInitThermal2_rule)
'''


def storageChargeRate_rule(model, t, out):
    return (model.Qin[t,out] <= model.maxStorCh[out] * model.StorageCap[out])
model.storageChargeRate = Constraint(model.Time, model.Out, rule=storageChargeRate_rule)

def storageDischRate_rule(model, t, out):
    return (model.Qout[t,out] <= model.maxStorDisch[out] * model.StorageCap[out])
model.storageDischRate = Constraint(model.Time, model.Out, rule=storageDischRate_rule)
'''
def storageMinState_rule(model, t, out):
    return (model.E[t,out] >= model.StorageCap[out] * model.minSoC[out])
model.storageMinState = Constraint(model.Time, model.Out, rule=storageMinState_rule)
'''

def storageCap_rule(model, t, out):
    return (model.E[t,out] <= model.StorageCap[out] )
model.storageCap = Constraint(model.Time, model.Out, rule=storageCap_rule)

#def storageMaxCap_rule(model, t, out):
#    return (model.E[t,out] <= model.maxStorageCap[out] )
#model.storageMaxCap = Constraint(model.Time, model.Out, rule=storageMaxCap_rule)


In [171]:
#-----------------------------------------------------------------------------#
## Objective functions
#-----------------------------------------------------------------------------#
def objective_rule(model):
    return (model.TotalCost)
model.Total_Cost = Objective(rule=objective_rule, sense=minimize)

#def objective_rule(model):
#    return (model.TotalCarbon2)
#model.Total_Carbon = Objective(rule=objective_rule, sense=minimize)

def opCost_rule(model):
    return(model.OpCost == ((sum (model.opPrices[inp] 
                            * sum(model.P[t,inp] for t in model.Time)
                            for inp in model.In)))
                            )
model.opCost = Constraint(rule=opCost_rule) 

def maintenanceCost_rule(model):
    return(model.MaintCost == (sum(model.P[t,inp] * 
                              model.cMatrix[inp,out] * model.omvCosts[inp]
                              for t in model.Time for inp in model.In for out in model.Out)
                              ))
model.maintCost = Constraint(rule=maintenanceCost_rule)

def incomeExp_rule(model):
    return(model.IncomeExp == (sum(model.feedInTariffs[out] * 
                            sum(model.Pexport[t,out] for t in model.Time) 
                            for out in model.Out))
                            )
model.incomeExp = Constraint(rule=incomeExp_rule)

def invCost_rule(model):
    return(model.InvCost == (sum(model.NetPresentValueTech[inp] * (model.linCapCosts[inp,out] * model.Capacities[inp,out] ) for inp in model.W_Ogrid for out in model.Out) #+ model.fixCapCosts[inp,out] * model.Ytechnologies[inp,out]
                            + sum(model.NetPresentValueStor[out] * model.linStorageCosts[out] * model.StorageCap[out] for out in model.Out))
                            )
model.invCost = Constraint(rule=invCost_rule) 

def totalCost_rule(model):
    return(model.TotalCost == model.InvCost + model.OpCost + model.MaintCost - model.IncomeExp)
model.totalCost = Constraint(rule=totalCost_rule) 

def totalCarbon2_rule(model):
    return(model.TotalCarbon2 == model.TotalCarbon)
model.totalCarbon2 = Constraint(rule=totalCarbon2_rule)

def totalCarbon_rule(model):
    return(model.TotalCarbon == sum(model.carbonFactors[inp] * sum(model.P[t,inp] for t in model.Time) for inp in model.In))
model.totalCarbon = Constraint(rule=totalCarbon_rule)


In [148]:
#-----------------------------------------------------------------------------#
## Print model ##
#-----------------------------------------------------------------------------#
#model.pprint()

#instance = model.create(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\generic_model.dat')


opt = SolverFactory("gurobi")
opt.options["mipgap"]=0.05
opt.options["FeasibilityTol"]=1e-05
solver_manager = SolverManagerFactory("serial")
#results = solver_manager.solve(instance, opt=opt, tee=True,timelimit=None, mipgap=0.1)

results = solver_manager.solve(model, opt=opt, tee=True,timelimit=None)

#sends results to stdout
#results.write()


def pyomo_save_results(options=None, instance=None, results=None):
    OUTPUT = open(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\Results_generic_hub.txt','w')
    print(results, file=OUTPUT)
    OUTPUT.close()
#pyomo_save_results(instance=instance, results=results)    

#Example of how to print variables
#print(instance.TotalCost.value)
print(model.TotalCost.value)




#for i in instance.Pnaturalgas:
#    print (instance.Pnaturalgas[i], instance.Pnaturalgas[i].value)

#print("--- %s seconds ---" % (time.time() - start_time))

## create the instance
#instance = model.create(’DiseaseEstimation.dat’)
## define the solver and its options
#solver = ’ipopt’
#opt = SolverFactory( solver )
#if opt is None:
#raise ValueError, "Problem constructing solver \
#‘"+str(solver)
#opt.set_options(’max_iter=2’)
## create the solver manager
#solver_manager = SolverManagerFactory( ’serial’ )
## solve
#results = solver_manager.solve(instance, opt=opt, tee=True,\
#timelimit=None)
#instance.load(results)
## display results
#display(instance)



Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Parameter mipgap unchanged
   Value: 0.05  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter FeasibilityTol to 1e-05
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Optimize a model with 359240 rows, 210305 columns and 1155791 nonzeros
Variable types: 183998 continuous, 26307 integer (26307 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Presolve removed 96449 rows and 43862 columns (presolve time = 24s) ...
Presolve removed 96449 rows and 43862 columns
Presolve time: 23.78s
Presolved: 262791 rows, 166443 columns, 727054 nonzeros
Variable types: 140163 continuous, 26280 integer (26280 binary)
Presolve removed 262791 rows and 166443 columns

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.2262600e+33 

In [149]:
print(model.TotalCarbon.value)

71903.8156271


In [206]:
Cap=model.Capacities.get_values()

In [18]:
model.Capacities.get_values()

{(1, 1): 5000.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (2, 1): 0.0,
 (2, 2): 27.293404094,
 (2, 3): 0.0,
 (3, 1): 0.0,
 (3, 2): 45.0,
 (3, 3): 0.0,
 (4, 1): 5.11751326763,
 (4, 2): 8.85329795299,
 (4, 3): 0.0,
 (5, 1): 0.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (6, 1): 0.0,
 (6, 2): 0.0,
 (6, 3): 0.0,
 (7, 1): 26.0115606936,
 (7, 2): 0.0,
 (7, 3): 45.0,
 (8, 1): 5.11751326763,
 (8, 2): 8.85329795299,
 (8, 3): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 45.0}

In [19]:
model.StorageCap.get_values()

{1: 0.0, 2: 0.0, 3: 0.0}

In [20]:
P_matrix=np.zeros(shape=(DemandData.shape[0],Technologies.shape[1]+1))
for i in range(1,DemandData.shape[0]+1):
    for j in range(1,Technologies.shape[1]+1+1):
        P_matrix[i-1,j-1]=model.P[i,j].value

In [21]:
P=pd.DataFrame(P_matrix)